In [ ]:
# https://www.kaggle.com/code/theoviel/dicom-resized-png-jpg

In [5]:
import os
import glob
from tqdm.auto import tqdm as tq
import pandas as pd
import numpy as np
import json
from joblib import Parallel, delayed

import tensorflow as tf


import cv2
import gdcm
import pydicom
import matplotlib.pyplot as plt

In [6]:
class CFG:
    def __init__(self):
        return

configs = CFG()
configs.dataset_dir = '/mnt/e/dataset/chexpert/CheXpert-v1.0'
configs.mode = 'train'
configs.data_dir = f"{configs.dataset_dir}/{configs.mode}.csv"

In [21]:
dataset_dir = '/mnt/e/dataset/chexpert/CheXpert-v1.0'
mode = 'train'


data = pd.read_csv(configs.data_dir)
data['Path'] = data['Path'].str.replace("CheXpert-v1.0", configs.dataset_dir)
data.head()


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,/mnt/e/dataset/chexpert/CheXpert-v1.0/train/pa...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,/mnt/e/dataset/chexpert/CheXpert-v1.0/train/pa...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,/mnt/e/dataset/chexpert/CheXpert-v1.0/train/pa...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,/mnt/e/dataset/chexpert/CheXpert-v1.0/train/pa...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,/mnt/e/dataset/chexpert/CheXpert-v1.0/train/pa...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [23]:
data.columns[5]

'No Finding'

In [22]:
data.values[:, 5:5+14]

array([[1.0, nan, nan, ..., nan, nan, 1.0],
       [nan, nan, -1.0, ..., nan, 1.0, nan],
       [nan, nan, nan, ..., nan, 1.0, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, 1.0, ..., nan, nan, 0.0],
       [1.0, nan, nan, ..., nan, nan, nan]], dtype=object)

In [19]:
images = data.Path.tolist()
images[:3]

['/mnt/e/dataset/chexpert/CheXpert-v1.0/train/patient00001/study1/view1_frontal.jpg',
 '/mnt/e/dataset/chexpert/CheXpert-v1.0/train/patient00002/study2/view1_frontal.jpg',
 '/mnt/e/dataset/chexpert/CheXpert-v1.0/train/patient00002/study1/view1_frontal.jpg']

In [20]:
img = cv2.imread(images[0])
img

array([[[  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1],
        ...,
        [  1,   1,   1],
        [  1,   1,   1],
        [  1,   1,   1]],

       ...,

       [[172, 172, 172],
        [158, 158, 158],
        [152, 152, 152],
        ...,
        [233, 233, 233],
        [200, 200, 200],
        [242, 242, 242]],

       [[163, 163, 163],
        [160, 160, 160],
        [137, 137, 137],
        ...,
        [239, 239, 239],
        [231, 231, 231],
        [255, 255, 255]],

       [[175, 175, 175],
        [162, 162, 162],
        [130, 130, 130],
        ...,
        [207, 207, 207],
        [205, 205, 205],
        [255, 255, 255]]